In [1]:
import re
from tqdm import tqdm
import numpy as npd
import pandas as pd

In [2]:
# тренируемся на input: лемма+тег output: словоформа, запоминая к какому классу(и роду) относится лемма 

### вычитываем парадигмы

In [2]:
%%time
indices = [0,2,3,4,5,6,8,9,10,11,12,13]  # интересующие столбцы в /odict.csv, 7,14,15 - 2locSg,2LocPl,2InsSg для F 'рукою'
sgNom,sgGen,sgDat,sgAcc,sgIns,sgLoc,plNom,plGen,plDat,plAcc,plIns,plLoc = [],[],[],[],[],[],[],[],[],[],[],[]
lemma = []
gender = []
paradigms = [lemma,gender,sgNom,sgGen,sgDat,sgAcc,sgIns,sgLoc,plNom,plGen,plDat,plAcc,plIns,plLoc]
noparadigm_file =  open('C:/hse_compling/diploma/data/odict/add_odict.csv', 'w', encoding = 'utf-8')
#pltnt_file =  open('C:/hse_compling/diploma/data/odict/add_odict_pltnt.csv', 'w', encoding = 'utf-8')
#sgtnt_file = open('C:/hse_compling/diploma/data/odict/add_odict_sgtnt.csv', 'w', encoding = 'utf-8')
with open('C:/hse_compling/diploma/data/odict/odict.csv', 'r', encoding = 'utf-8') as file:
    for k,line in enumerate(file):
        try:
            sline = line.strip().split(',')
            if sline[1] not in ['ж', 'м', 'мо', 'с', 'жо', 'мн.']:
                continue
            else:
    #дубли, второе условие нужно, потому что есть вхождения типа "микроб мо" и "микроб м", оставляем оба для мерджа с df_classes
                if sline[0] in lemma and sline[1] == gender[lemma.index(sline[0])]: 
                    continue
                elif sline[1] in ['ж', 'м', 'мо', 'с', 'жо', 'мн.'] and len(sline) == 3:
                    noparadigm_file.write(line)
                    continue
                elif sline[1] == 'мн.' and  len(sline) > 7: # pluralia tantum
                    #pltnt_file.write(','.join([sline[0],sline[2],sline[3],sline[4],sline[5],sline[6]])) #plNom,plGen,plDat,plAcc,plIns,plLoc
                    #pltnt_file.write('\n')
                    sline[2:8] = ['-']*6 #заменяем формы ед.числа на '-'
                elif sline[1] in ['ж', 'м', 'мо', 'с', 'жо'] and 7 < len(sline) < 14: # singularia tantum
                    #sgtnt_file.write(','.join([sline[0],sline[2],sline[3],sline[4],sline[5],sline[6]])) #sgNom,sgGen,sgDat,sgAcc,sgIns,sgLoc
                    #sgtnt_file.write('\n')
                    sline = sline [:8] + ['-']*7 # дописываем '-' вместо форм мн.числа                
                    #elif len(sline) >= 14: #эти уже посчитали
                gender.append(sline[1])            
                for i in range(12):
                    index = indices[i]
                    paradigms[i+2].append(sline[index])
                sline[0] = sline[0].replace('ё','е')
                lemma.append(sline[0])

        except Exception as e:
            print(e)
            print(k,line)
noparadigm_file.close()


Wall time: 1min 5s


In [3]:
#датафрейм с формами
columns = ["lemma","gender", "form1N","form1G","form1D","form1A","form1I","form1L","form2N","form2G","form2D","form2A","form2I","form2L"]
data = list(zip(columns, paradigms))
df_paradigms = pd.DataFrame.from_items(data)

In [4]:
df_paradigms.shape

(49342, 14)

### удаляем дубликаты

In [5]:
for i,k in enumerate(df_paradigms.duplicated(['lemma','gender'])):
    if k == True:
        print(df_paradigms.iloc[[i-1],[0,1]])
        print(df_paradigms.iloc[[i],[0,1]])

         lemma gender
2478  балясник     мо
         lemma gender
2479  балясник     мо
        lemma gender
3176  береста      ж
        lemma gender
3177  береста      ж
      lemma gender
3760  бобер     мо
      lemma gender
3761  бобер     мо
          lemma gender
3880  болванчик     мо
          lemma gender
3881  болванчик     мо
         lemma gender
4208  бревешко      с
         lemma gender
4209  бревешко      с
       lemma gender
5011  вахтер     мо
       lemma gender
5012  вахтер     мо
       lemma gender
10860  дрема      ж
       lemma gender
10861  дрема      ж
        lemma gender
11292  еканье      с
        lemma gender
11293  еканье      с
         lemma gender
11469  железка      ж
         lemma gender
11470  железка      ж
         lemma gender
12297  заметка      ж
         lemma gender
12298  заметка      ж
      lemma gender
12821   зев      м
      lemma gender
12822   зев      м
           lemma gender
13057  знаменщик     мо
           lemma gender
1305

In [6]:
df_paradigms = df_paradigms.drop_duplicates(subset = ['lemma','gender'])

In [7]:
df_paradigms.shape

(49302, 14)

In [8]:
df_paradigms[:5]

,lemma,gender,form1N,form1G,form1D,form1A,form1I,form1L,form2N,form2G,form2D,form2A,form2I,form2L
0,авиакрыло,с,авиакрыло,авиакрыла,авиакрылу,авиакрыло,авиакрылом,авиакрыле,авиакрылья,авиакрыльев,авиакрыльям,авиакрылья,авиакрыльями,авиакрыльях
1,антикрыло,с,антикрыло,антикрыла,антикрылу,антикрыло,антикрылом,антикрыле,антикрылья,антикрыльев,антикрыльям,антикрылья,антикрыльями,антикрыльях
2,барин,мо,барин,барина,барину,барина,барином,барине,баре,бар,барам,бар,барами,барах
3,беремя,с,беремя,беремени,беремени,беремя,беременем,беремени,-,-,-,-,-,-
4,бесенок,мо,бесёнок,бесёнка,бесёнку,бесёнка,бесёнком,бесёнке,бесенята,бесенят,бесенятам,бесенят,бесенятами,бесенятах


In [9]:
df_paradigms.loc[df_paradigms['lemma'] == 'полет']

,lemma,gender,form1N,form1G,form1D,form1A,form1I,form1L,form2N,form2G,form2D,form2A,form2I,form2L
30262,полет,м,полёт,полёта,полёту,полёт,полётом,полёте,полёты,полётов,полётам,полёты,полётами,полётах


### создаем датафрейм с леммой-классом

In [10]:

genders = [] #колонка нужна для мерджа с df_paradigms
lemmas = []
classtags = [] 
''' 0 - все формы, 1 -pl.tantum, 2 - sg.tantum.
если pl.tantum,то в gender сохраняем 'мн.', чтобы совпадало c df_paradigms['gender']
classtag дописываем род 
вирши 2 мн. м 4а: classtag = м4а+ gender = мн.
щегольство 10 с 1в- : classtag = c1в- gender = c
sg.tantum иногда пишется как [-], запоминаем, чтобы потом дополнить тэг и удалить нагенеренные формы
'''
notes = open('C:/hse_compling/diploma/data/odict/notes.csv', 'w', encoding = 'utf-8')
#создаем словарь для кодирования регулярных отклонений в окончаниях
notes_dict = {}
notes_dict[''] = '@' 
i = 0
#with open('C:/hse_compling/diploma/data/odict/small.txt', 'r', encoding = 'utf-8') as file:#
with open('C:/hse_compling/diploma/data/odict/zalizniak_ordered.txt', 'r', encoding = 'utf-8') as file:
    for line in file: #надо проверить, существительное ли это, и выделить из возможных пояснений окончания и пометы
        notespart = ''
        number = ''
        l = line.split('@')
        lemmapart = l[0]
        if len(l) == 2:
            notespart = l[1]
        if "[-]" in lemmapart or re.search('мн\..*затрудн\.',lemmapart) or lemmapart[0].isupper():
            number = '-'        
        lemma_tags = re.split("[\(\_\@\{\[\#\$]|, [а-яА-Яё]|=>|<=", lemmapart)[0]
        lemma_tags = re.split("\s+",lemma_tags)
        lemma_tags = [i for i in lemma_tags if i]
        if len(lemma_tags)< 4 or (lemma_tags[0] in lemmas) or lemma_tags[-2] not in ['ж', 'м', 'мо', 'с', 'жо']:
            continue
        elif lemma_tags[-3] == 'мн.':
            number = '+'
            lemma, gender, realgender, classtag = lemma_tags[0],lemma_tags[-3],lemma_tags[-2],lemma_tags[-1]
        elif lemma_tags[-2] in ['ж', 'м', 'мо', 'с', 'жо']:
            lemma, gender, realgender, classtag = lemma_tags[0],lemma_tags[-2],lemma_tags[-2],lemma_tags[-1]           
        #из части с пояснениями достаем особенности окончаний, кодируем их, чтобы добавить в тег
        if len(l) == 2:
            notes.write(line)
            r = re.search("[а-яА-Я]+\, \-[а-яА-Я]+, \-[а-яА-Я]+", notespart)
            if r:
                match = r.group(0)
            else:
                match = ""
            if match not in notes_dict:
                i += 1
                notes_dict[match] = '@' + str(i)  
                #print(match,notes_dict[match])
            note = notes_dict[match]
        else:
            note = ''
            
        classtag = classtag.rstrip(',')
        classtag = realgender + classtag + number + note
        classtag.replace('--','-')
        genders.append(gender)
        lemmas.append(lemma)
        classtags.append(classtag)

In [11]:
columns = ["lemma","gender","classtag"]
dataset = list(zip(lemmas, genders, classtags))
df_classes = pd.DataFrame(data = dataset, columns = columns)

In [12]:
df_classes.loc[df_classes['lemma']=='Кинешма']

,lemma,gender,classtag
1144,Кинешма,ж,ж1а-


In [13]:
for i,k in enumerate(df_classes.duplicated('lemma')):
    if k == True:
        print(df_classes.iloc[i-1]['lemma'], df_classes.iloc[i-1]['gender'], df_classes.iloc[i-1]['classtag'])
        print(df_classes.iloc[i]['lemma'], df_classes.iloc[i]['gender'], df_classes.iloc[i]['classtag'])

### merge

In [14]:

df = pd.merge(df_paradigms, df_classes, on=['lemma','gender'])

In [15]:
df.shape

(46766, 15)

In [16]:
columns = list(df.columns)

In [17]:
columns = columns[:2] + columns[-1:] + columns[2:-1]

In [18]:
df = df[columns]

In [19]:
df[:5]

,lemma,gender,classtag,form1N,form1G,form1D,form1A,form1I,form1L,form2N,form2G,form2D,form2A,form2I,form2L
0,авиакрыло,с,с1D@1,авиакрыло,авиакрыла,авиакрылу,авиакрыло,авиакрылом,авиакрыле,авиакрылья,авиакрыльев,авиакрыльям,авиакрылья,авиакрыльями,авиакрыльях
1,антикрыло,с,с1D@1,антикрыло,антикрыла,антикрылу,антикрыло,антикрылом,антикрыле,антикрылья,антикрыльев,антикрыльям,антикрылья,антикрыльями,антикрыльях
2,барин,мо,мо1**а@,барин,барина,барину,барина,барином,барине,баре,бар,барам,бар,барами,барах
3,беремя,с,с8**с-,беремя,беремени,беремени,беремя,беременем,беремени,-,-,-,-,-,-
4,бесенок,мо,мо3**а@2,бесёнок,бесёнка,бесёнку,бесёнка,бесёнком,бесёнке,бесенята,бесенят,бесенятам,бесенят,бесенятами,бесенятах


In [20]:
df['lemma'] = df['form1N'] # теперь леммы тоже с ё

In [21]:
for i in tqdm(range(df.shape[0])): #удаляем лишние формы для singularia и pluralia tantum 
    if df.loc[df.index[i], 'classtag'].endswith('+'):
        for k in range(3,9):
            df.iloc[i,k] = '-'
    elif df.loc[df.index[i], 'classtag'].endswith('-'):
        for k in range(9,15):
            df.iloc[i,k] = '-'

100%|████████████████████████████████████████████| 46766/46766 [00:07<00:00, 6260.16it/s]


In [22]:
df.loc[df['lemma']=='корма']

,lemma,gender,classtag,form1N,form1G,form1D,form1A,form1I,form1L,form2N,form2G,form2D,form2A,form2I,form2L
16293,корма,ж,ж1в-,корма,кормы,корме,корму,кормой,корме,-,-,-,-,-,-


In [23]:
df.loc[df['lemma']=='дрожжи']

,lemma,gender,classtag,form1N,form1G,form1D,form1A,form1I,form1L,form2N,form2G,form2D,form2A,form2I,form2L
10335,дрожжи,мн.,м4е+,-,-,-,-,-,-,дрожжи,дрожжей,дрожжам,дрожжи,дрожжами,дрожжах


In [24]:
#save to csv
df.to_csv('data/data.csv', sep='\t', encoding = 'utf-8', index=False)
#save to excel
writer = pd.ExcelWriter('data/data.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

In [25]:

df = pd.read_csv('data/data.csv', sep='\t', encoding = 'utf-8')

### сохраняем  lowercase

In [26]:
df.loc[df['lemma']=='Гибралтар']

,lemma,gender,classtag,form1N,form1G,form1D,form1A,form1I,form1L,form2N,form2G,form2D,form2A,form2I,form2L
7444,Гибралтар,м,м1а-,Гибралтар,Гибралтара,Гибралтару,Гибралтар,Гибралтаром,Гибралтаре,-,-,-,-,-,-


In [27]:
%%time
for i in range(df.shape[0]):
    for k in range(15):
        if k != 2:
            df.iloc[i,k] = df.iloc[i,k].lower()
    

Wall time: 2min 59s


In [28]:
df.loc[df['lemma']=='башка']

,lemma,gender,classtag,form1N,form1G,form1D,form1A,form1I,form1L,form2N,form2G,form2D,form2A,form2I,form2L
2670,башка,ж,ж3*в-,башка,башки,башке,башку,башкой,башке,-,-,-,-,-,-


In [29]:
df = df.drop_duplicates(subset = ['lemma','gender'])

In [30]:
#save to csv
df.to_csv('data/data_lower.csv', sep='\t', encoding = 'utf-8', index=False)
#save to excel
writer = pd.ExcelWriter('data/data_lower.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

### short format to long format

In [31]:
df = pd.read_csv('data/data_lower.csv', sep='\t', encoding = 'utf-8')

In [32]:
df[:5]

,lemma,gender,classtag,form1N,form1G,form1D,form1A,form1I,form1L,form2N,form2G,form2D,form2A,form2I,form2L
0,авиакрыло,с,с1D@1,авиакрыло,авиакрыла,авиакрылу,авиакрыло,авиакрылом,авиакрыле,авиакрылья,авиакрыльев,авиакрыльям,авиакрылья,авиакрыльями,авиакрыльях
1,антикрыло,с,с1D@1,антикрыло,антикрыла,антикрылу,антикрыло,антикрылом,антикрыле,антикрылья,антикрыльев,антикрыльям,антикрылья,антикрыльями,антикрыльях
2,барин,мо,мо1**а@,барин,барина,барину,барина,барином,барине,баре,бар,барам,бар,барами,барах
3,беремя,с,с8**с-,беремя,беремени,беремени,беремя,беременем,беремени,-,-,-,-,-,-
4,бесёнок,мо,мо3**а@2,бесёнок,бесёнка,бесёнку,бесёнка,бесёнком,бесёнке,бесенята,бесенят,бесенятам,бесенят,бесенятами,бесенятах


In [34]:
%%time
df_long = pd.wide_to_long(df, stubnames = 'form', i = ['lemma','gender'], j = 'formtag')

Wall time: 1.51 s


In [35]:
df_long.shape

(560640, 2)

In [36]:
df_long = df_long.sample(frac=1, random_state=15)

In [37]:
df_long[:10]

,,,classtag,form
lemma,gender,formtag,,
календарь,м,2D,м2в,календарям
никонианин,мо,2L,мо1**а,никонианах
уилкания,ж,1I,ж7а-,уилканией
инстанция,ж,2D,ж7а,инстанциям
скверность,ж,1D,ж8а,скверности
цепня,ж,1D,ж2*в,цепне
лингвист,мо,1A,мо1а,лингвиста
контрреволюционер,мо,2A,мо1а,контрреволюционеров
картография,ж,1G,ж7а,картографии


In [38]:
%%time
#save to csv
df_long.to_csv('data/data_lower_long.csv', sep='\t', encoding = 'utf-8')
#save to excel
writer = pd.ExcelWriter('data/data_lower_long.xlsx')
df_long.to_excel(writer,'Sheet1')
writer.save()

Wall time: 5min 47s
